# Dataset fit_partial
---
- Dataset을 fit_partial 할 필요가 있다.
- Dataset에는 기본적으로 rating.csv 부분은 바뀌지 않는다!

최초 fit( 
    users : 전체 사용자 (가변) - 신규 사용자 들어올때마다 해당 index를 늘려줘야한다. => 아래로 붙여야할듯
    items: 전체 위스키 ids (불변)
    user_features : 사용 X
    item_features : cost, flavor만 사용



rating.csv  => interactions 변하지 않는다.
하지만, 
db에서 가져오는 user-review-whisky 는 변할 수 있다.



In [1]:
import os
import pandas as pd
import numpy as np
from time import time
from tqdm.notebook import tqdm
import random
import pickle

from scipy.sparse import coo_matrix, csr_matrix

from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k

from hyperopt import fmin, hp, tpe, Trials

from sklearn.base import clone

## Normalization
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
random.seed(402)

In [3]:
class LightFMResizable(LightFM):
    """A LightFM that resizes the model to accomodate new users,
    items, and features"""

    def fit_partial(
        self,
        interactions,
        user_features=None,
        item_features=None,
        sample_weight=None,
        epochs=1,
        num_threads=1,
        verbose=False,
    ):
        try:
            self._check_initialized()
            self._resize(interactions, user_features, item_features)
        except ValueError:
            # This is the first call so just fit without resizing
            pass

        super().fit_partial(
            interactions,
            user_features,
            item_features,
            sample_weight,
            epochs,
            num_threads,
            verbose,
        )

        return self

    def _resize(self, interactions, user_features=None, item_features=None):
        """Resizes the model to accommodate new users/items/features"""

        no_components = self.no_components
        no_user_features, no_item_features = interactions.shape  # default

        if hasattr(user_features, "shape"):
            no_user_features = user_features.shape[-1]
        if hasattr(item_features, "shape"):
            no_item_features = item_features.shape[-1]

        if (
            no_user_features == self.user_embeddings.shape[0]
            and no_item_features == self.item_embeddings.shape[0]
        ):
            return self

        new_model = clone(self)
        new_model._initialize(no_components, no_item_features, no_user_features)

        # update all attributes from self._check_initialized
        for attr in (
            "item_embeddings",
            "item_embedding_gradients",
            "item_embedding_momentum",
            "item_biases",
            "item_bias_gradients",
            "item_bias_momentum",
            "user_embeddings",
            "user_embedding_gradients",
            "user_embedding_momentum",
            "user_biases",
            "user_bias_gradients",
            "user_bias_momentum",
        ):
            # extend attribute matrices with new rows/cols from
            # freshly initialized model with right shape
            old_array = getattr(self, attr)
            old_slice = [slice(None, i) for i in old_array.shape]
            new_array = getattr(new_model, attr)
            new_array[tuple(old_slice)] = old_array
            setattr(self, attr, new_array)

        return self

In [4]:
# item_features = pd.read_csv("dataset/min_max_item_features.csv", index_col=0, encoding="UTF-8")
# user_features = pd.read_csv("dataset/min_max_user_features.csv", index_col=0, encoding="UTF-8")
# rating = pd.read_csv("dataset/rating.csv", index_col=0, encoding="UTF-8")
# whisky = pd.read_csv("dataset/whisky.csv", index_col=0, encoding="UTF-8")

rating = pd.read_csv("../dataset/rating.csv", index_col=0, encoding="UTF-8")
whisky = pd.read_csv("../dataset/whisky.csv", index_col=0, encoding="UTF-8")

In [5]:
cols = whisky.columns.tolist()
cols = [cols[8]] + cols[11:]
cols

['price_tier',
 'smoky',
 'peaty',
 'spicy',
 'herbal',
 'oily',
 'body',
 'rich',
 'sweet',
 'salty',
 'vanilla',
 'tart',
 'fruity',
 'floral']

In [ ]:
item_df = whisky[cols]

In [ ]:
# Define the scalers
min_max_scaler = MinMaxScaler()

In [ ]:
# Apply Min-Max normalization
min_max_data = min_max_scaler.fit_transform(whisky[numeric_cols])

In [ ]:
min_max_data = pd.DataFrame(min_max_data, columns=numeric_cols)
min_max_data

In [ ]:
item_df[cols[1:]] = min_max_scaler.fit_transform(item_df[cols[1:]])

In [ ]:
item_df